
Last Edit Date: 2024-07-26

Description: Moves XML files from one Google Cloud storage folder to another. 

In [ ]:
import os
from google.cloud import storage

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/user/ficc/ficc/mitas_creds.json'

In [ ]:
STORAGE_CLIENT = storage.Client()

In [ ]:
def move_file(source_bucket_name: str, source_folder: str, file_name: str, destination_bucket_name: str, destination_folder: str):
    # Get the source and destination buckets
    source_bucket = STORAGE_CLIENT.bucket(source_bucket_name)
    destination_bucket = STORAGE_CLIENT.bucket(destination_bucket_name)

    # Construct the full source and destination blob names
    source_blob_name = os.path.join(source_folder, file_name)
    destination_blob_name = os.path.join(destination_folder, file_name)

    # Get the source blob
    source_blob = source_bucket.blob(source_blob_name)

    # Check if the source blob exists
    if not source_blob.exists():
        print(f'The file {source_blob_name} does not exist in the source bucket.')
        return None
    
    # Copy the blob to the destination bucket
    destination_blob = destination_bucket.blob(destination_blob_name)
    destination_blob.rewrite(source_blob)

    # Delete the original blob
    source_blob.delete()

    print(f'Moved {source_blob_name} to {destination_blob_name}')

In [ ]:
def list_files_with_prefix(bucket_name: str, folder: str, prefix: str = '') -> list:
    # Initialize a client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # List blobs with the specified prefix in the specified folder
    blobs = bucket.list_blobs(prefix=os.path.join(folder, prefix))

    return [blob.name for blob in blobs]

In [ ]:
SP_REF_DATA_BUCKET_NAME = 'sp_ref_data'
SOURCE_FOLDER_PATH = '2024/07/02/'
DESTINATION_FOLDER_PATH = '2024/07/01'

In [ ]:
files = list_files_with_prefix(SP_REF_DATA_BUCKET_NAME, SOURCE_FOLDER_PATH, 'Muni_Standard-20240701')

In [ ]:
for file in files:
    move_file(SP_REF_DATA_BUCKET_NAME, SOURCE_FOLDER_PATH, os.path.basename(file), SP_REF_DATA_BUCKET_NAME, DESTINATION_FOLDER_PATH)    # `os.path.basename(...)` removes the directory